In [2]:
from utils import get_boot_sector, get_fat_info, get_block

In [3]:
with open('/Users/jackbullen/Downloads/subdirs.img', "rb") as f:
        # Get super block and fat.
        os_name, block_size, block_count, fat_start, fat_blocks, root_dir_start, root_dir_blocks = get_boot_sector(f)
        fat = get_fat_info(f, fat_start, fat_blocks, block_size)

In [4]:
def get_directory(f, block_size, dir_start, dir_blocks):
    directory_files = dict()

    # Go to start of dir and read until the end.
    f.seek(dir_start * block_size, 0)
    for i in range(dir_blocks*512 // 64):
        status = int.from_bytes(f.read(1), 'big')
        starting_block = int.from_bytes(f.read(4), 'big')
        num_blocks = int.from_bytes(f.read(4), 'big')
        file_size = int.from_bytes(f.read(4), 'big')
        year = int.from_bytes(f.read(2), 'big')
        month = int.from_bytes(f.read(1), 'big')
        day = int.from_bytes(f.read(1), 'big')
        hour = int.from_bytes(f.read(1), 'big')
        minute = int.from_bytes(f.read(1), 'big')
        second = int.from_bytes(f.read(1), 'big')
        creation_time = f"{year:04d}/{month:02d}/{day:02d} {hour:02d}:{minute:02d}:{second:02d}"
        modification_time = int.from_bytes(f.read(7), 'big')
        filename = f.read(31).decode("utf-8")
        filler = f.read(6)

        directory_entry = {
            'status':status, 
            'filename': filename.strip('\x00'), 
            'start_block': starting_block, 
            'num_blocks': num_blocks, 
            'file_size': file_size, 
            'creation_time': creation_time, 
            'modification_time': modification_time
        }
        
        if status == 5:
            sub_dir_start = starting_block
            sub_dir_blocks = num_blocks
            item = get_directory(f, block_size, sub_dir_start, sub_dir_blocks)

            directory_files[filename.strip('\x00')] = item

        elif status == 3:
            # pass
            directory_files[filename.strip('\x00')] = directory_entry

    return directory_files

In [5]:
def get_directory(f, block_size, dir_start, dir_blocks) -> dict:
    '''
        Return root directory
    '''
    root_items = list()

    # go to start of root and read until the end.
    f.seek(dir_start * block_size, 0)
    for i in range(dir_blocks*block_size // 64):
        status = int.from_bytes(f.read(1), 'big')
        starting_block = int.from_bytes(f.read(4), 'big')
        num_blocks = int.from_bytes(f.read(4), 'big')
        file_size = int.from_bytes(f.read(4), 'big')
        year = int.from_bytes(f.read(2), 'big')
        month = int.from_bytes(f.read(1), 'big')
        day = int.from_bytes(f.read(1), 'big')
        hour = int.from_bytes(f.read(1), 'big')
        minute = int.from_bytes(f.read(1), 'big')
        second = int.from_bytes(f.read(1), 'big')
        creation_time = f"{year:04d}/{month:02d}/{day:02d} {hour:02d}:{minute:02d}:{second:02d}"
        modification_time = int.from_bytes(f.read(7), 'big')
        filename = f.read(31).decode("utf-8")
        filler = f.read(6)

        # skip the No File entries
        if status == 0:
            continue

        root_item = {
            'status':status, 
            'filename': filename.strip('\x00'), 
            'start_block': starting_block, 
            'num_blocks': num_blocks, 
            'file_size': file_size, 
            'creation_time': creation_time, 
            'modification_time': modification_time
        }

        root_items.append(root_item)

    return root_items

In [6]:
for i in range(root_dir_blocks*block_size // 64):
        print(int.from_bytes(file[:1], 'big'))
        # status = int.from_bytes(file[1:2], 'big')
        # starting_block = int.from_bytes(file[2:6], 'big')
        # num_blocks = int.from_bytes(file[6:10], 'big')
        # file_size = int.from_bytes(file[10:14], 'big')
        # year = int.from_bytes(file[14:16], 'big')
        # month = int.from_bytes(file[16:17], 'big')
        # day = int.from_bytes(file[17:18], 'big')
        # hour = int.from_bytes(file[18:19], 'big')
        # minute = int.from_bytes(file[19:20], 'big')
        # second = int.from_bytes(file[20:21], 'big')
        # creation_time = f"{year:04d}/{month:02d}/{day:02d} {hour:02d}:{minute:02d}:{second:02d}"
        # modification_time = int.from_bytes(file[21:28], 'big')
        # filename = file[28:59].decode("utf-8")
        # filler = file[59:64]
        file = file[64:]

NameError: name 'file' is not defined

In [7]:
def get_directory(dir_bytes, block_size) -> dict:
    '''
        Return root directory
    '''
    entries = list()
    for _ in range(root_dir_blocks*block_size // 64):
            status = int.from_bytes(dir_bytes[0:1], 'big')
            starting_block = int.from_bytes(dir_bytes[1:5], 'big')
            num_blocks = int.from_bytes(dir_bytes[5:9], 'big')
            file_size = int.from_bytes(dir_bytes[9:13], 'big')
            year = int.from_bytes(dir_bytes[13:15], 'big')
            month = int.from_bytes(dir_bytes[15:16], 'big')
            day = int.from_bytes(dir_bytes[16:17], 'big')
            hour = int.from_bytes(dir_bytes[17:18], 'big')
            minute = int.from_bytes(dir_bytes[18:19], 'big')
            second = int.from_bytes(dir_bytes[19:20], 'big')
            creation_time = f"{year:04d}/{month:02d}/{day:02d} {hour:02d}:{minute:02d}:{second:02d}"
            modification_time = int.from_bytes(dir_bytes[20:27], 'big')
            file_name = dir_bytes[27:58].decode("utf-8")
            filler = dir_bytes[58:64]
            dir_bytes = dir_bytes[64:]

            if status == 0:
                continue

            entries.append({
                'status':status, 
                'file_name': file_name.strip('\x00'), 
                'start_block': starting_block, 
                'num_blocks': num_blocks, 
                'file_size': file_size, 
                'creation_time': creation_time, 
                'modification_time': modification_time
            })
    return entries

In [8]:
def get_linked_blocks(f, num_blocks, block_size, dir_start, fat) -> bytes:
    '''
        Return the linked blocks of a file or directory.
    '''
    curr = dir_start
    file = b''
    for _ in range(num_blocks):
        block_data = get_block(f, curr, block_size)
        file += block_data
        curr = fat[curr]
    return file

In [16]:
def get_subdirs(f, file_system, subdirs, parent):
    '''
        Return the file system containing all subdirectories and files inside of subdirs.

        - Recursively called to get all subdirectories and files inside of subdirs.
    '''
    
    for subdir in subdirs:
        subdir_blocks = get_linked_blocks(f, subdir['num_blocks'], block_size, subdir['start_block'], fat)
        subdir_entries = get_directory(subdir_blocks, block_size)
        print(subdir_entries)
        file_entries = [file for file in subdir_entries if file['status'] == 3]
        subdir_subdirs = [dir for dir in subdir_entries if dir['status'] == 5]

        files = {}
        for file in file_entries:
            files[file['file_name']] = get_linked_blocks(f, file   ['num_blocks'], block_size, file['start_block'], fat)

        file_system[parent+subdir['file_name']] = {
                                'entries': subdir_entries,
                                'files': files
                            }

        if subdir_entries:
            file_system = get_subdirs(f, file_system, subdir_subdirs, parent+subdir['file_name']+'/')

    return file_system

In [17]:
def get_file_system():
    file_system = dict()
    with open('/Users/jackbullen/Downloads/subdirs.img', "rb") as f:
        root_blocks = get_linked_blocks(f, root_dir_blocks, block_size, root_dir_start, fat)
        root_entries = get_directory(root_blocks, block_size)
        file_entries = [file for file in root_entries if file['status'] == 3]
        subdir_entries = [dir for dir in root_entries if dir['status'] == 5]

        files = {}
        for file in file_entries:
            files[file['file_name']] = get_linked_blocks(f, file   ['num_blocks'], block_size, file['start_block'], fat)

        file_system['/'] = {
                                'entries': root_entries,
                                'files': files
                            }

        file_system = get_subdirs(f, file_system, subdir_entries, '/')

    return file_system

FS = get_file_system()

[{'status': 5, 'file_name': 'subdir2', 'start_block': 84, 'num_blocks': 4, 'file_size': 2048, 'creation_time': '2005/11/15 12:00:00', 'modification_time': 2204568310775808}]
[{'status': 3, 'file_name': 'foo.txt', 'start_block': 61, 'num_blocks': 5, 'file_size': 2560, 'creation_time': '2005/11/15 12:00:00', 'modification_time': 2204568310775808}]


In [18]:
FS

{'/': {'entries': [{'status': 3,
    'file_name': 'mkfile.cc',
    'start_block': 1,
    'num_blocks': 2,
    'file_size': 735,
    'creation_time': '2005/11/15 12:00:00',
    'modification_time': 2204568310775808},
   {'status': 5,
    'file_name': 'subdir1',
    'start_block': 80,
    'num_blocks': 4,
    'file_size': 2048,
    'creation_time': '2005/11/15 12:00:00',
    'modification_time': 2204568310775808},
   {'status': 3,
    'file_name': 'disk.img.gz',
    'start_block': 66,
    'num_blocks': 8,
    'file_size': 3940,
    'creation_time': '2009/08/04 21:11:13',
    'modification_time': 2208953288428301}],
  'files': {'mkfile.cc': b'//\n// mkfile.cc\n//\n\n#include <stdio.h>\n#include <iostream>\n\n#define BUFFER_SIZE\t512\n#define BLOCK_COUNT\t5\n\nusing namespace std;\n\nint main ( int argc, char *argv[] )\n{\n\tchar *buffer[BUFFER_SIZE];\n\tint\tblockCount = BLOCK_COUNT;\n\tchar ch\t= \'A\';\n\t\n\tif ( argc != 4 )\n\t{\n\t\tcout << "Usage: makefile filename size character" <